In [1]:
pip install pandas
#install pandas if not already installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import my csv file from github and then print the head to make sure it imported ok/looks ok

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/oheag2/UCDPA_GillianOHea/main/work_life.csv?token=GHSAT0AAAAAABUP7HD2AIWNDM2PG44FIPB2YU5YCPA')

print(df.head())